In [1]:
import pandas as pd
import folium
import numpy as np
import random
import geopandas as gpd

from random import shuffle
from shapely.geometry import Polygon, Point
from sklearn.cluster import DBSCAN
from sklearn import metrics

### Read data from firebase

In [2]:
import firebase_admin
from firebase_admin import credentials

if not firebase_admin._apps:
    cred = credentials.Certificate('/firebase_apikey.json') 
    default_app = firebase_admin.initialize_app(cred, {'databaseURL': "https://firebase_db.firebaseio.com"})

In [3]:
# Import database module.
from firebase_admin import db

# Get a database reference to our posts
ref = firebase_admin.db.reference("users")

# Read the data at the posts reference (this is a blocking operation)
print(ref.get())

{'-MIE0wiYRMhXTpl4fRRe': {'Latitude': 48.137321, 'Longitude': 11.575555}, '-MIE1aGP6aahUm3Il7mM': {'Latitude': 48.137321, 'Longitude': 11.575555}, '-MIE2LxILKungei8KjG3': {'Latitude': 48.13727899999935, 'Longitude': 11.575554500000576}, '-MIE2M0GURSdSoQD5d2u': {'Latitude': 48.13732099999925, 'Longitude': 11.575155500000166}, '-MIE2M4VUEJf0dhY5Prl': {'Latitude': 48.13719699999956, 'Longitude': 11.575191500000203}, '-MIE2M8-cQ5HezXOZX7O': {'Latitude': 48.137036499999965, 'Longitude': 11.575004500000011}, '-MIE2MBcfKTyCWOj-cCe': {'Latitude': 48.13731099999927, 'Longitude': 11.575040500000048}, '-MIE2MEoPfj1lQI0Sppa': {'Latitude': 48.13731549999926, 'Longitude': 11.575493500000514}, '-MIE2MITKAM8pz11L4Ut': {'Latitude': 48.13724949999943, 'Longitude': 11.575428500000447}, '-MIE2MLx_ZS2z5AjKmmH': {'Latitude': 48.1371409999997, 'Longitude': 11.575138500000149}, '-MIE2MPsQFtX5iDz2oCw': {'Latitude': 48.13711699999976, 'Longitude': 11.575552500000574}, '-MIE2MU99bKQ7_UgJSM9': {'Latitude': 48.137

In [6]:
# convert to pandas df
df = pd.DataFrame(ref.get()).T
df.head()

,Latitude,Longitude
-MIE0wiYRMhXTpl4fRRe,48.137321,11.575555
-MIE1aGP6aahUm3Il7mM,48.137321,11.575555
-MIE2LxILKungei8KjG3,48.137279,11.575555
-MIE2M0GURSdSoQD5d2u,48.137321,11.575156
-MIE2M4VUEJf0dhY5Prl,48.137197,11.575192


In [7]:
latlon = list(zip(df.Latitude, df.Longitude))
latlon

[(48.137321, 11.575555),
 (48.137321, 11.575555),
 (48.13727899999935, 11.575554500000576),
 (48.13732099999925, 11.575155500000166),
 (48.13719699999956, 11.575191500000203),
 (48.137036499999965, 11.575004500000011),
 (48.13731099999927, 11.575040500000048),
 (48.13731549999926, 11.575493500000514),
 (48.13724949999943, 11.575428500000447),
 (48.1371409999997, 11.575138500000149),
 (48.13711699999976, 11.575552500000574),
 (48.137313999999265, 11.575831500000861),
 (48.137226999999484, 11.57590900000094),
 (48.13714649999969, 11.575043500000051),
 (48.13720599999954, 11.575598000000621),
 (48.13710499999979, 11.575194000000206),
 (48.13712699999974, 11.575447000000466),
 (48.13726849999938, 11.575410000000428),
 (48.137253999999416, 11.5751880000002),
 (48.137290499999324, 11.575876000000907),
 (48.13718549999959, 11.57595700000099),
 (48.13707099999988, 11.5752855000003),
 (48.137194999999565, 11.575028500000036),
 (48.137222999999494, 11.575678000000703),
 (48.13724799999943, 11.57

In [36]:
# visualize points on map
mapit = folium.Map( location=[48.137323, 11.575593], zoom_start=15)
for coord in latlon:
    folium.Marker( location=[ coord[0], coord[1] ], fill_color='#43d9de', radius=2 ).add_to( mapit )   
    
#mapit.save('all_coords.html')
mapit

### Clustering - DBSCAN

In [8]:
# the epsilon roughly 2 meters
kms_per_radian = 6371.0088
epsilon = 0.002 / kms_per_radian

# Run the DBSCAN from sklearn
coords = pd.DataFrame(latlon, columns=["Latitude", "Longitude"])
db = DBSCAN(eps=epsilon, min_samples=3, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))

cluster_labels = db.labels_
n_clusters = len(set(cluster_labels))

# get the cluster
# cluster_labels = -1 means outliers
clusters = pd.Series([coords[cluster_labels == n] for n in range(-1, n_clusters)])

In [9]:
len(clusters)

61

In [10]:
#Plotting all points 
from folium.plugins import MarkerCluster 
from folium import plugins 
some_map_1 = folium.Map(location=[coords['Latitude'].mean(), coords['Longitude'].mean()], zoom_start=25) 
mc = MarkerCluster() 
j=0 
row=0 
for row in range(len(clusters)): 
    for j in clusters[row].index: 
        mc.add_child(folium.Marker(location=[clusters[row].loc[j][0],clusters[row].loc[j][1]], popup=row)) 
    some_map_1.add_child(mc) 
some_map_1 

### Find clusters with violations 
- = number of people in cluster > max allowed according to official regulations (ex. 50)

In [12]:
from collections import Counter
Counter(cluster_labels)

Counter({0: 19,
         1: 7,
         2: 28,
         3: 8,
         4: 18,
         5: 7,
         6: 16,
         7: 150,
         27: 48,
         8: 4,
         9: 7,
         10: 32,
         38: 3,
         11: 4,
         12: 31,
         13: 20,
         14: 27,
         15: 29,
         16: 18,
         17: 3,
         18: 14,
         19: 6,
         -1: 1879,
         26: 13,
         20: 16,
         21: 13,
         22: 7,
         23: 32,
         24: 8,
         25: 11,
         36: 4,
         28: 5,
         29: 16,
         30: 11,
         31: 4,
         32: 5,
         33: 6,
         34: 4,
         35: 4,
         37: 3,
         56: 3,
         39: 3,
         45: 3,
         40: 3,
         41: 4,
         54: 3,
         44: 3,
         42: 3,
         43: 4,
         48: 3,
         46: 4,
         47: 3,
         50: 3,
         49: 3,
         51: 3,
         52: 3,
         53: 3,
         57: 3,
         58: 3,
         55: 3})

In [13]:
from collections import Counter
for k, v in dict(Counter(cluster_labels)).items():
    if k != -1: # skip outliers (class of isolated points)
        if int(v) > 50:
            print("Alert! " + str(v) + " persons are violationg physical distancing rules!")
            print("Cluster number: " + str(k+1)) # plus one because of "-1" outliers class
            print("Locations list:")
            print(clusters[k+1])

Alert! 150 persons are violationg physical distancing rules!
Cluster number: 8
Locations list:
       Latitude  Longitude
9     48.137141  11.575139
13    48.137146  11.575044
15    48.137105  11.575194
16    48.137127  11.575447
21    48.137071  11.575286
...         ...        ...
1886  48.137035  11.575126
1920  48.137104  11.575289
1938  48.137060  11.575363
1963  48.137134  11.575196
1989  48.137097  11.575086

[150 rows x 2 columns]


In [14]:
# visualize (violating cluster)
k = 8 
m = folium.Map( location=[clusters[k].Latitude.mean(), clusters[k].Longitude.mean()], zoom_start=40)
for coord in list(zip(clusters[k].Latitude, clusters[k].Longitude)):
    folium.Marker( location=[ coord[0], coord[1] ], fill_color='#43d9de', radius=2 ).add_to(m) 
m